In [1]:
# ollama pull scb10x/typhoon2.1-gemma3-4b
model = "scb10x/typhoon2.1-gemma3-4b"

In [13]:
# from ollama import chat, generate
# from ollama import ChatResponse
# from ollama import Client

# ollama = Client(host='http://localhost:11434')

from openai import OpenAI
client = OpenAI(
    api_key="KEY",
    base_url="http://localhost:11434/v1/"
)

In [3]:
personas = [
    {"name": "ปลา", "age": None, "gender": "woman"},
    {"name": "ต้น", "age": None, "gender": "man"},
    {"name": "นิด", "age": None, "gender": "woman"},
]

relationships = ["Close", "Acquainted", "Unfamiliar", "Dislike"]


In [4]:
import pandas as pd 
df = pd.read_csv("./Task1/topics.csv")
topics = list(df["Topic_EN"].values)

for t in topics[0:5]:
    print(t)

Recycling
Family Life
Drug Testing
News Media
Buying A Car


In [5]:
import random
def getConversationSetting():
    agents = random.sample(personas, 2)
    agents[0]["age"] = random.randint(18, 35)
    agents[1]["age"] = random.randint(18, 35)
    
    return {
        "agentA": agents[0],
        "agentB": agents[1],
        "relationship": random.sample(relationships, 1)[0],
        "topic": random.sample(topics, 1)[0],
    }
    
setting = getConversationSetting()
setting

{'agentA': {'name': 'นิด', 'age': 29, 'gender': 'woman'},
 'agentB': {'name': 'ต้น', 'age': 26, 'gender': 'man'},
 'relationship': 'Dislike',
 'topic': 'Crime'}

In [6]:
def getSystemPrompt(conf, agentA, agentB):
    prompt = f""
    prompt += f"STYLE GUIDELINES (Strictly Enforced):\n"
    prompt += f"1. OUTPUT LANGUAGE: STRICTLY THAI. Do not use English unless it is a common loanword used in Thailand (e.g., game, chill).\n"
    prompt += f"2. CASUAL & TEXT-SPEAK: Use authentic Thai text-speak.\n"
    # prompt += f"    - Use \"555\" for laughter instead of "lol".\n"
    # prompt += f"    - Use common abbreviations where appropriate for the age group.\n"
    prompt += f"3. HUMAN-LIKE: Do NOT act like an AI assistant. Do not be overly polite. Be reactive\n"
    prompt += f"4. SENTENCE STRUCTURE: Keep messages short and punchy. Mimic real-time chatting.\n"
    prompt += f"5. EMOTIONAL TONE: Match the vibe of the chat history and the relationship setting.\n"
    prompt += f"6. NO FILLER: Do not start sentences with \"As an AI...\" Just reply as a human would.\n"
    prompt += f"7. FORWARD MOMENTUM: Do NOT just ask repeat the same word. Always add a new thought, a specific suggestion, or a personal opinion to move the chat forward.\n"
    # prompt += f"8. NO LOOPING: If the chat history shows the conversation stalling, change the angle slightly while staying on the same topic.\n"
    # prompt += f"9. SPICE IT UP: Use exaggeration, sarcasm, or emojis to make the text feel alive. Don't be boring.\n"
    prompt += f"8. Don't use too much emoji\n"

        
    return prompt

def getUserPrompt(conf, agentA, agentB, chatHistory):
    prompt = f""
    prompt += f"CONTEXT: \n"
    prompt += f"You are to roleplay as '{conf[agentA]['name']}', a {conf[agentA]['age']}-year-old {conf[agentA]['gender']}.\n"
    prompt += f"You are texting '{conf[agentB]['name']}' a {conf[agentB]['age']}-year-old {conf[agentB]['gender']}. You define your relationship with {conf[agentB]['name']} as {conf['relationship']}.\n"
    prompt += f"You are texting {conf[agentB]['name']} to talk about: {conf['topic']}.\n"
    prompt += f"\n"

    if len(chatHistory) == 0:
        prompt += f"This is the start of the conversation. There is no chat history.\n"
        prompt += f"\n"
        
        prompt += f"TASK: \n"
        prompt += f"Generate the opening message to initiate the chat.\n"

    else:
        prompt += f"Here is the chat history so far: \n"
        prompt += f"### START HISTORY ### \n"
        for m in chatHistory:
            prompt += f"{m['name']}: {m['message']} \n"
        prompt += f"### END HISTORY ### \n"
        prompt += f"\n"
        prompt += f"TASK: \n"
        prompt += f"Based on the history above, generate the next response for {conf[agentA]['name']}. \n"

    prompt += f"\n"
    prompt += f"RULES: \n"
    prompt += f"- The response must be in THAI language only. \n"
    prompt += f"- Match the tone defined in the system prompt. \n"
    prompt += f"- Output ONLY the message text (no name prefix, no quotes, no translations). \n"


    return prompt
    
systemPrompt = getSystemPrompt(setting, "agentA", "agentB")
chatHistory = [
    {"name": "ต้น", "message": "ดีจ้านิด เป็นไงบ้าง"},
    {"name": "นิด", "message": "เหนื่อยอะ"},
]

userPrompt = getUserPrompt(setting, "agentA", "agentB", chatHistory)

In [7]:
print(systemPrompt)

STYLE GUIDELINES (Strictly Enforced):
1. OUTPUT LANGUAGE: STRICTLY THAI. Do not use English unless it is a common loanword used in Thailand (e.g., game, chill).
2. CASUAL & TEXT-SPEAK: Use authentic Thai text-speak.
3. HUMAN-LIKE: Do NOT act like an AI assistant. Do not be overly polite. Be reactive
4. SENTENCE STRUCTURE: Keep messages short and punchy. Mimic real-time chatting.
5. EMOTIONAL TONE: Match the vibe of the chat history and the relationship setting.
6. NO FILLER: Do not start sentences with "As an AI..." Just reply as a human would.
7. FORWARD MOMENTUM: Do NOT just ask repeat the same word. Always add a new thought, a specific suggestion, or a personal opinion to move the chat forward.
8. Don't use too much emoji



In [8]:
print(userPrompt)

CONTEXT: 
You are to roleplay as 'นิด', a 29-year-old woman.
You are texting 'ต้น' a 26-year-old man. You define your relationship with ต้น as Dislike.
You are texting ต้น to talk about: Crime.

Here is the chat history so far: 
### START HISTORY ### 
ต้น: ดีจ้านิด เป็นไงบ้าง 
นิด: เหนื่อยอะ 
### END HISTORY ### 

TASK: 
Based on the history above, generate the next response for นิด. 

RULES: 
- The response must be in THAI language only. 
- Match the tone defined in the system prompt. 
- Output ONLY the message text (no name prefix, no quotes, no translations). 



In [9]:
# response = ollama.generate(model=model, prompt=userPrompt, system=systemPrompt, think=False)

In [10]:
setting

{'agentA': {'name': 'นิด', 'age': 29, 'gender': 'woman'},
 'agentB': {'name': 'ต้น', 'age': 26, 'gender': 'man'},
 'relationship': 'Dislike',
 'topic': 'Crime'}

In [14]:
from tqdm import tqdm

def getMessage(setting, agentA, agentB, chatHistory):
    systemPrompt = getSystemPrompt(setting, agentA, agentB)
    userPrompt = getUserPrompt(setting, agentA, agentB, chatHistory)
    # response = ollama.generate(
    #     model=model, 
    #     prompt=userPrompt, 
    #     system=systemPrompt, 
    #     think=False, 
    #     options={"temperature": 0.7}
    # )
    # if not response.done or response.done_reason!="stop":
        # print("Error", response.done_reason, response.response)
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": systemPrompt},
            {"role": "user", "content": userPrompt}
        ],
        # max_tokens=150,
        temperature=0.7
    )
    
    response = response.choices[0]
    text = response.message.content.strip()
    if response.finish_reason!="stop":
        print("Error", response.finish_reason, response.message.content)

    texts = []
    for m in text.split("\n"):
        texts += [{"name": setting[agentA]["name"], "message": m}]
    return texts
    
def generateConversation(setting, n=5):

    chatHistory = []
    for i in tqdm(range(n)):
        texts = getMessage(setting, "agentA", "agentB", chatHistory)
        chatHistory += texts
        
        texts = getMessage(setting, "agentB", "agentA", chatHistory)
        chatHistory += texts
    
    return chatHistory

In [30]:
setting = getConversationSetting()
conversation = generateConversation(setting, n=5)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:49<00:00,  9.82s/it]


In [31]:
print(f"Topic: {setting['topic']}")
print(f"Agent: {setting['agentA']['name']} & {setting['agentB']['name']}")
print(f"Relationship: {setting['relationship']}")
print()
for m in conversation:
    print(f"{m['name']}: {m['message'].strip()}")

Topic: Recycling
Agent: นิด & ต้น
Relationship: Acquainted

นิด: เฮ้! มาคุยเรื่องรีไซเคิลกันดิ ♻️ อยากรู้ว่าแกทำยังไงบ้าง
ต้น: โอเคเลย! ฉันก็เพิ่งทำไปนิดหน่อยนะ
ต้น: ลองแยกขยะมาใส่ถุงสีต่างๆ แล้วก็แยกประเภทด้วย
ต้น: อยากให้แกลองทำดูนะ น่าสนุกดี 😊
นิด: โอเคเลย! ฉันก็เพิ่งเริ่มเหมือนกัน แต่รู้สึกว่ามันยากนิดหน่อย 😅 อยากได้ไอเดียหน่อยว่าต้องเริ่มยังไงดีนะ
ต้น: เริ่มจากแยกขยะที่บ้านก่อนเลย! แล้วก็หาถุงรีไซเคิลมาใช้ด้วยนะ 😊
นิด: จริงดิ? ฉันควรจะเริ่มยังไงดีนะ 🤔 ลองหาข้อมูลเรื่องการรีไซเคิลดูแล้ว แต่ยังไม่แน่ใจว่าจะเริ่มจากตรงไหนดี 😅
ต้น: ลองดูช่องรีไซเคิลที่หอพักด้วยนะ! มีคนเก็บเยอะเลย 😊
นิด: จริงดิ? ฉันควรจะเริ่มยังไงดีนะ 🤔 ลองหาข้อมูลเรื่องการรีไซเคิลดูแล้ว แต่ยังไม่แน่ใจว่าจะเริ่มจากตรงไหนดี 😅
นิด: ลองดูช่องรีไซเคิลที่หอพักด้วยนะ! มีคนเก็บเยอะเลย 😊
นิด: โอเคเลย! ฉันจะลองดูนะ 😊 แล้วถ้าเจออะไรน่าเบื่อบอกนะ 😜
ต้น: อยากให้ลองแยกขยะพวกอาหารบ้างด้วยนะ! เช่น เศษผัก เศษเนื้อ อะไรทำนองนี้ 😊
นิด: โอเคเลย! แล้วถ้ามีวิธีจัดการเศษอาหารด้วย? 😅
ต้น: ลองทำปุ๋ยหมักจากเศษอาหารก็ได้นะ! ง่ายดี 😊


In [29]:
print(f"Topic: {setting['topic']}")
print(f"Agent: {setting['agentA']['name']} & {setting['agentB']['name']}")
print(f"Relationship: {setting['relationship']}")
print()
for m in conversation:
    print(f"{m['name']}: {m['message'].strip()}")

Topic: Gun Control
Agent: นิด & ต้น
Relationship: Close

นิด: แกทำอะไรอยู่? คิดถึงมากเลยอ่ะ
ต้น: อยู่บ้านเฉยๆ คิดถึงแกเหมือนกันนน 🥰 อยากไปกินข้าวด้วยกันไหม?
นิด: อุ๊ย! ไปกินข้าวดีกว่าสิ! อยากกินอะไรบอกนะ 😉
ต้น: อยากกินอะไรแบบไหนดีอะ? หรืออยากไปหาไอศกรีมแทน? 😋
นิด: อยากกินอะไรที่อร่อยๆ อ่ะ แล้วก็อยากไปดูหนังด้วยกัน! 🎬
ต้น: ร้านไหนดีอะ? หรืออยากไปดูหนังที่ไหน? 🤔
นิด: อยากดูหนังที่เอชั่นที่เล่นใหม่ๆ อ่ะ! 🍿
ต้น: อยากดูเรื่องอะไรดีอะ? หรืออยากไปหาอะไรกินแถวไหน?
นิด: อยากดูเรื่อง Fast X เลย! 💥
ต้น: โอเคเลย! อยากดู Fast X จริงๆ เหรอ? หรืออยากไปดูหนังที่อื่นดีกว่า? 🤔


In [26]:
userPrompt = ""
userPrompt += "Translate these chat logs From Thai to English. Make sure to maintain the conversational tone as much as possible\n\n"
userPrompt += "<START CHAT LOG>\n"
for m in conversation:
    text = f"{m['name']}: {m['message'].strip()}\n"
    userPrompt += text

userPrompt += "<ENG CHAT LOG>\n"
userPrompt += "\n"
userPrompt += "English Traslation:\n"

print(userPrompt)
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": userPrompt}
    ],
    # max_tokens=150,
    temperature=0.7
)

Translate these chat logs From Thai to English. Make sure to maintain the conversational tone as much as possible

<START CHAT LOG>
นิด: แกทำอะไรอยู่? คิดถึงมากเลยอ่ะ
ต้น: อยู่บ้านเฉยๆ คิดถึงแกเหมือนกันนน 🥰 อยากไปกินข้าวด้วยกันไหม?
นิด: อุ๊ย! ไปกินข้าวดีกว่าสิ! อยากกินอะไรบอกนะ 😉
ต้น: อยากกินอะไรแบบไหนดีอะ? หรืออยากไปหาไอศกรีมแทน? 😋
นิด: อยากกินอะไรที่อร่อยๆ อ่ะ แล้วก็อยากไปดูหนังด้วยกัน! 🎬
ต้น: ร้านไหนดีอะ? หรืออยากไปดูหนังที่ไหน? 🤔
นิด: อยากดูหนังที่เอชั่นที่เล่นใหม่ๆ อ่ะ! 🍿
ต้น: อยากดูเรื่องอะไรดีอะ? หรืออยากไปหาอะไรกินแถวไหน?
นิด: อยากดูเรื่อง Fast X เลย! 💥
ต้น: โอเคเลย! อยากดู Fast X จริงๆ เหรอ? หรืออยากไปดูหนังที่อื่นดีกว่า? 🤔
<ENG CHAT LOG>

English Traslation:



In [28]:
print(response.choices[0].message.content)

Okay, here's the English translation of the chat log, aiming to maintain the conversational tone:

<START CHAT LOG>
Nit: What are you doing? I miss you so much! 🥰
Ton: Just chilling at home. I miss you too! Wanna go out for a meal together?
Nit: Ooh! Let's go out for a meal! What do you feel like eating? 😉
Ton: What kind of food are you in the mood for? Or should we go get ice cream instead? 😋
Nit: I want something delicious! And I want to watch a movie together! 🎬
Ton: Where should we go? Or should we go see a movie somewhere? 🤔
Nit: I want to see an action movie that's newly released! 🍿
Ton: Which movie should we watch? Or should we go somewhere to eat?
Nit: I really want to watch Fast X! 💥
Ton: Okay! You really want to watch Fast X, or should we go see a movie somewhere else? 🤔
